# Week 3 - Lab

This lab consists of a couple different exercises. First, we'll build a couple neural networks. In the second exercise we'll execute a provided CloudFormation template.

**Table of Contents**

-   [Exercise 1](#exercise-1)
    -   [Cloud Formation](#cloud-formation)
-   [Exercise 2](#exercise-2)
-   [Deliverables](#deliverables)

<a id='exercise-1'></a>
## Exercise 1

<a id='cloud-formation'></a>
### Cloud Formation

Run the following Cloudformation template to create an EC2 instance that exposes a Jupyter Hub server.

```YAML
AWSTemplateFormatVersion: '2010-09-09'
Description: 'GPU-based data science toolbox with RStudio Server'
Resources:
  GPUInstance:
    Type: 'AWS::EC2::Instance'
    Properties:
      InstanceType: 'p2.xlarge'
      ImageId: 'ami-0174e69c12bae5410'
      NetworkInterfaces: 
        - AssociatePublicIpAddress: true
          DeviceIndex: 0
          GroupSet:
            - Ref: CS4315Summer2019SecurityGroupLab3Instructor
          SubnetId: 'subnet-09e1bc3eb37714763'
      UserData: 
        Fn::Base64: !Sub |
          #!/bin/bash

          # Install system-wide tensorflow
          /usr/local/bin/pip3 install tensorflow-gpu==1.10.0

          # R
          echo "deb http://cran.rstudio.com/bin/linux/ubuntu xenial/" | sudo tee -a /etc/apt/sources.list
          gpg --keyserver keyserver.ubuntu.com --recv-key E298A3A825C0D65DFD57CBB651716619E084DAB9
          gpg -a --export E298A3A825C0D65DFD57CBB651716619E084DAB9 | sudo apt-key add -
          apt-get update
          apt-get -y install r-base

          # Install system-wide R Packages
          R -e 'install.packages("devtools")'
          R -e 'devtools::install_github("rstudio/reticulate"); devtools::install_github("rstudio/keras")'
          R -e 'library(keras); install_keras(tensorflow="gpu")'
          
          # R Studio Server
          apt-get -y install gdebi-core
          wget https://download2.rstudio.org/server/trusty/amd64/rstudio-server-1.2.1335-amd64.deb
          gdebi --non-interactive rstudio-server-1.2.1335-amd64.deb

          # User Setup
          users="eric.eckstrand blah.blah"
          for i in $users; do useradd -m -d /home/$i -s /bin/bash $i; done
          for i in $users; do usermod --password $(openssl passwd -1 $i) $i; done

          # Include CUDA libraries in shared library location
          sed -i '1iLD_LIBRARY_PATH="/usr/local/cuda/lib64"' /usr/lib/R/etc/ldpaths
          ldconfig
          ldconfig /usr/local/cuda/lib64
          ldconfig /usr/local/cuda/lib

          # Restart RStudio Server
          rstudio-server stop
          rstudio-server start
      Tags:
        -
          Key: Name
          Value: 'CS4315Summer2019-Lab3-Instructor'
      KeyName: 'CS4315Summer2019-Key-Share'

  CS4315Summer2019SecurityGroupLab3Instructor:
    Type: AWS::EC2::SecurityGroup
    Properties:
      GroupDescription: Inbound and outbound rules for CS4315 GPU EC2 instance
      SecurityGroupEgress:
      - CidrIp: '0.0.0.0/0'
        Description: All
        FromPort: 0
        IpProtocol: tcp
        ToPort: 65535
      SecurityGroupIngress:
      - CidrIp: '0.0.0.0/0'
        Description: All
        FromPort: 0
        IpProtocol: tcp
        ToPort: 65535
      Tags:
      - Key: Name
        Value: 'CS4315Summer2019-Lab3SecurityGroup-Instructor'
      VpcId: 'vpc-00b2b0d347ea0e3cc'
```

<a id='bonus'></a>
## Bonus (Optional)

For 10 extra points, repeat the Brazilian Coins (Classification) steps from the lecture on the R Studio instance you created above, or some other R Studio instance. Use the `keras` library for R. `Warning` keras on R can be troublesome to install. Good luck!  Provide your code and output in a text document.

<a id='deliverables'></a>
## Deliverables

- Your Cloudformation template for Exercise 1. You should add me as an RStudio user. I will log onto your instance to check that it works.